In [1]:
import sympy as sp
import pandas as pd
from operator import mul
from functools import reduce

In [2]:
def print_derivatives(nodes, *args):
    index = nodes
    cols = ['f' + i * '\'' for i in range(len(derivatives_per_node[0]))]
    return pd.DataFrame(derivatives_per_node, index=index, columns=cols)

In [3]:
def calculate_div_diffs(nodes, *args, max_order=None):
    derivatives_per_node = list(zip(*args))
    max_order_per_node = [len([j for j in i if j != None]) for i in derivatives_per_node]
    if max_order == None:
        max_order = sum(max_order_per_node)
        
    nodes_with_order = [nodes[i] for i in range(len(nodes)) for j in range(max_order_per_node[i])]
    values_with_order = [args[0][i] for i in range(len(nodes)) for j in range(max_order_per_node[i])]
    node_to_pos = {n: p for p, n in enumerate(nodes)}
    nwo_to_pos = {i: node_to_pos[nodes_with_order[i]] for i in range(len(nodes_with_order))}
    
    diffs = [values_with_order]
    for order in range(max_order - 1):
        current_diffs = []
        for i in range(1, len(nodes_with_order) - order):
            if nodes_with_order[i + order] == nodes_with_order[i - 1]:
                current_diffs.append(
                    (derivatives_per_node[nwo_to_pos[i]][order + 1]) / 
                    (reduce(mul, [1] + [j for j in range(1, order + 1)]))
                )
            else:
                p = diffs[-1][i] - diffs[-1][i-1]
                current_diffs.append((diffs[-1][i] - diffs[-1][i-1]) / (nodes_with_order[i + order] - nodes_with_order[i-1]))
        if len(current_diffs) == 0:
            break
        diffs.append(current_diffs)
    return diffs

In [4]:
def print_diffs(diffs):
    rows = 2 * len(diffs[0]) - 1
    for r in range(rows):
        row = '\t' if r % 2 == 1 else ''
        for i in range(r % 2, min(rows - r, min(r + 1, len(diffs))), 2):
            row += f'{float(diffs[i][(r - i) // 2]):.3f}\t\t'
        print(row[:-2])

In [5]:
nodes = [-1, 1, 2, 3]
fvals = [7, -11, -5, 43]
dfvals = [None, 5, 1, None]
ddfvals = [None, -22, None, None]

In [6]:
derivatives_per_node = list(zip(fvals, dfvals, ddfvals))
max_order_per_node = [len([j for j in i if j != None]) for i in derivatives_per_node]
nodes_with_order = [nodes[i] for i in range(len(nodes)) for j in range(max_order_per_node[i])]

In [7]:
print_derivatives(nodes, fvals, dfvals, ddfvals)

,f,f',f''
-1,7,NaN,NaN
1,-11,5.0,-22.0
2,-5,1.0,NaN
3,43,NaN,NaN


In [8]:
dds = calculate_div_diffs(nodes, fvals, dfvals, ddfvals)

In [9]:
print_diffs(dds)

7.000
	-9.000
-11.000		7.000
	5.000		-14.500
-11.000		-22.000		12.500
	5.000		23.000		-13.833
-11.000		1.000		-29.000		9.083
	6.000		-6.000		22.500
-5.000		-5.000		16.000
	1.000		26.000
-5.000		47.000
	48.000
43.000


In [10]:
x = sp.symbols('x')
polynom = 0
mult = 1
for i in range(len(dds)):
    polynom += dds[i][0] * mult
    mult *= x - nodes_with_order[i]

In [11]:
sp.expand(polynom)

9.08333333333333*x**6 - 68.3333333333333*x**5 + 176.833333333333*x**4 - 149.333333333333*x**3 - 87.9166666666667*x**2 + 208.666666666667*x - 100.0

In [14]:
polynom_test = []
dp = polynom
for i in range(len(derivatives_per_node[0])):
    f_i = []
    for j in range(len(nodes)):
        if derivatives_per_node[j][i] != None:
            f_i.append(dp.evalf(subs={x: nodes[i]}))
        else:
            f_i.append(None)
    dp = sp.diff(dp)
    polynom_test.append(f_i)

In [19]:
print_derivatives(nodes, *polynom_test)

,f,f',f''
-1,7,NaN,NaN
1,-11,5.0,-22.0
2,-5,1.0,NaN
3,43,NaN,NaN
